In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset, get_basic_model_config
import numpy as np

In [3]:
data = Dataset(name='mortality', dirname='preprocess/MIMIC-Mortality/', labelfield='hospmort1yr')

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [14]:
from models.rnnmodel import Model as RNNModel
from models.rnnmodel_probing import Model as RNNProbingModel

In [5]:
from trainer import ClassificationTrainer, ClassificationEvaluator

Mortality Normal
================

In [16]:
train_data = data.get_data('train')
test_data = data.get_data('test')

Pos Percentage 0.23886764103264582
Pos Percentage 0.2388569866010391


In [ ]:
mortality_config = get_basic_model_config(data, 'mortality.test.MLP')
trainer = ClassificationTrainer(RNNModel, mortality_config)
trainer.train(train_data, test_data)

Setting Embedding


In [ ]:
evaluator = ClassificationEvaluator(RNNModel, 'outputs/lstm_mortality/SunDec2322:44:562018_MLP/')
pred = evaluator.evaluate(test_data)

Gender Normal
=============

In [ ]:
gender_train_data = data.get_data_for_probing('gender', 'train')
gender_test_data = data.get_data_for_probing('gender', 'test')

In [ ]:
gender_config = data.get_basic_model_config('gender')
gender_config['training']['lr'] = 0.0001
trainer = ClassificationTrainer(RNNModel, gender_config, model_name='MLP')
trainer.train(gender_train_data, gender_test_data, average='micro')

In [ ]:
gender_classifier = ClassificationEvaluator(RNNModel, dirname='outputs/lstm_main_gender/WedDec2601:05:392018_MLP/')
gender_pred = gender_classifier.evaluate(gender_test_data)

Gender Prober
=============

In [ ]:
gender_config = data.get_basic_model_config('gender', prober=True)
trainer = ClassificationTrainer(RNNProbingModel, gender_config, model_name='MLP', exp_name='mortality')
trainer.model.load_encoder_values('outputs/lstm_mortality/TueDec2521:27:122018_MLP/')
trainer.train(gender_train_data, gender_test_data, average='micro')

In [ ]:
gender_probe_classifier = ClassificationEvaluator(RNNProbingModel, dirname='outputs/lstm_probing_gender/mortality/MonDec2417:23:092018_MLP/')
pred = gender_probe_classifier.evaluate(gender_test_data)

In [ ]:
eval_params = dict(evaluator.model.encoder.named_parameters())
gender_params = dict(gender_probe_classifier.model.prober.named_parameters())
for n, v in gender_params.items() :
    if n in eval_params :
        print(n, (v != eval_params[n]).sum())
    else :
        print(n, 'not in evaluator')

Gender Prober Random
====================

In [ ]:
gender_config = data.get_basic_model_config('gender.test.MLP.random_init', prober=True)
trainer = ClassificationTrainer(RNNProbingModel, gender_config)
trainer.train(gender_train_data, gender_test_data, average='micro')

In [ ]:
random_classifier = ClassificationEvaluator(RNNProbingModel, dirname='outputs/lstm_probing_gender/random/MonDec2419:07:092018_MLP/')
pred = random_classifier.evaluate(gender_test_data)

In [ ]:
eval_params = dict(evaluator.model.encoder.named_parameters())
gender_params = dict(random_classifier.model.prober.named_parameters())
for n, v in gender_params.items() :
    if n in eval_params :
        print(n, (v != eval_params[n]).sum())
    else :
        print(n, 'not in evaluator')

Gender Majority
===============

In [ ]:
from sklearn.metrics import classification_report
gy = gender_test_data.y
from pandas.io.json.normalize import nested_to_record    

flat = nested_to_record(classification_report(gy, [1]*len(gy), output_dict=True), sep='/')
flat.update({'acc' : 1.0})
# print(accuracy_score(gy, [1]*len(gy)))
# print(roc_auc_score(gy, [sum(gy)/len(gy)]*len(gy)))
# print(roc_auc_score(gy, [1]*len(gy)))

In [ ]:
str(flat)